# Pinecone Assistant - Context snippets

The [Pinecone Assistant](https://docs.pinecone.io/guides/assistant/overview) `context_assistant` operation lets you directly retrieve relevant information snippets for your queries. Using the same set of files uploaded to Assistant, you can now alternate between `.chat()` and `.context()` requests and use Pinecone Assistant in different ways across different layers of your application stack.

## Motivation

The `context_assistant` operation is designed to enable two capabilities: 1) enable the use of Assistant with different LLMs (e.g., Llama 3.1) and 2) directly access retrieved relevant context information without a model reasoning step. The operation provides all the benefits of Assistant-- including parsing, chunking, and indexing on upload, as well as query planning, searching, and result aggregation on query—- while bypassing LLM reasoning and the associated transformation of the results.

## 1. Assistant setup

Install the Pinecone Python SDK and import the packages.

In [ ]:
%pip install -qU \
     pinecone==7.0.2 \
     pinecone-notebooks==0.1.1

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import json

from pinecone import Pinecone
from pinecone_plugins.assistant.models import Message

### Get and set the Pinecone API key

We will need a free [Pinecone API key](https://docs.pinecone.io/guides/assistant/quickstart). The code below will either authenticate you and set the API key as an environment variable or will prompt you to enter the API key and then set it in the environment.

In [5]:
from getpass import getpass

def get_pinecone_api_key():
    """
    Get Pinecone API key from environment variable or prompt user for input.
    Returns the API key as a string.

    Only necessary for notebooks. When using Pinecone yourself, 
    you can use environment variables or the like to set your API key.
    """
    api_key = os.environ.get("PINECONE_API_KEY")
    
    if api_key is None:
        try:
            # Try Colab authentication if available
            from pinecone_notebooks.colab import Authenticate
            Authenticate()
            # If successful, key will now be in environment
            api_key = os.environ.get("PINECONE_API_KEY")
        except ImportError:
            # If not in Colab or authentication fails, prompt user for API key
            print("Pinecone API key not found in environment.")
            api_key = getpass("Please enter your Pinecone API key: ")
            # Save to environment for future use in session
            os.environ["PINECONE_API_KEY"] = api_key
            print("Pinecone API key saved to environment.")
            
    return api_key

PINECONE_API_KEY = get_pinecone_api_key()

Pinecone API key not found in environment.
Pinecone API key saved to environment.


### Create a Pinecone Assistant

In [6]:
# Set Assistant name
assistant_name = "my-cool-context-assistant"

pc = Pinecone(
    # defaults to os.environ.get("PINECONE_API_KEY")
    # or uncomment and add the API key inline
    # api_key = "my_pinecone_api_key"
)
assistants_list = pc.assistant.list_assistants()

# Check that this Assistant does not already exist, if so, use it.

if assistant_name not in [a.name for a in assistants_list]:
    assistant = pc.assistant.create_assistant(assistant_name)
else:
    assistant = pc.assistant.Assistant(assistant_name=assistant_name)

assistant

{'created_at': '2025-07-03T20:16:46.372597382Z',
 'host': 'https://prod-1-data.ke.pinecone.io',
 'instructions': None,
 'metadata': {},
 'name': 'my-cool-context-assistant',
 'status': 'Ready',
 'updated_at': '2025-07-03T20:16:48.275068083Z'}

### Download an example PDF

We will use Netflix Annual Report (10K) for 2016 that is available for download online [here](https://d18rn0p25nwr6d.cloudfront.net/CIK-0001065280/7c37ae9c-ace5-4ad5-ab02-6b5e82c5ff8f.pdf).

In [9]:
!curl -O "https://d18rn0p25nwr6d.cloudfront.net/CIK-0001065280/7c37ae9c-ace5-4ad5-ab02-6b5e82c5ff8f.pdf" > netflix-10k.pdf 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  564k  100  564k    0     0  1307k      0 --:--:-- --:--:-- --:--:-- 1310k


## 2. Retrieve context

Now, let's upload the PDF file to Assistant, and run some queries against it to get context snippets.

### Upload the file we downloaded to Pinecone Assistant

You can monitor this upload in the Assistant Console here: https://app.pinecone.io/organizations/-/projects/-/assistant/my-cool-context-assistant/files.

In [10]:
file_names = [f.name for f in assistant.list_files()]

file_name = "netflix-10k.pdf"

if file_name not in file_names:

  response = assistant.upload_file(
      file_path=file_name,
      timeout=None,
      metadata={ "published": "2016-01-28", "document_type": "10k" },
  )
  print(response)
else:
  print(f"file {file_name} already uploaded")

file netflix-10k.pdf already uploaded


In [11]:
# We can also list the files on Assistant to validate the file exists
assistant.list_files()

[FileModel(name='netflix-10k.pdf', id='4fecb1e0-3874-4706-8d7e-2962574401a9', metadata={'document_type': '10k', 'published': '2016-01-28'}, created_on='2025-07-03T20:36:49.488104873Z', updated_on='2025-07-03T20:37:01.426460790Z', status='Available', percent_done=1.0, signed_url=None, error_message=None, size=578489.0)]

### Run a context query

In [12]:
response = assistant.context(
    query="Who is the Chief Financial Officer of Netflix?",
    # Uncomment to filter by metadata
    # filter={ "document_type": { "$eq": "10k" } }
)

print(response.snippets)

[Snippet(type='text', content='ures to be designed under our supervision, to\r\nensure that material information relating to the registrant, including its consolidated subsidiaries, is made known to us by others within those\r\nentities, particularly during the period in which this report is being prepared;\r\nb) designed such internal control over financial reporting, or caused such internal control over financial reporting to be designed under our\r\nsupervision, to provide reasonable assurance regarding the reliability of financial reporting and the preparation of financial statements for\r\nexternal purposes in accordance with generally accepted accounting principles;\r\nc) evaluated the effectiveness of the registrant’s disclosure controls and procedures and presented in this report our conclusions about the\r\neffectiveness of the disclosure controls and procedures, as of the end of the period covered by this report based on such evaluation; and\r\nd) disclosed in this report any

### Analyze the response
The response object returning from Assistant has 2 important fields: `snippets` and `usage`. First let's fetch the relevant `snippets` and explore them.

In [13]:
# Snippets are ordered by relevancy (most relevant first)
most_relevant_snippet = response.snippets[0]

# content is the actuall text data stored in the snippet
content = most_relevant_snippet.content

# Let's make sure David Wells, Netflix CFO in 2016 appears in the snippet
print(f'"David Wells" in content - {"David Wells" in content}')

"David Wells" in content - True


### Use context references

In [14]:
# reference is the document and page the snippet comes from
reference = most_relevant_snippet.reference

# Let'e print the document name and the relevant page:
print(f"The snippets comes from document: {reference.file.name} on pages: {reference.pages}")

The snippets comes from document: netflix-10k.pdf on pages: [70, 71, 72]


### Deep dive - context response

Let's review the context object returning with more details:
there are 3 main keys: `snippet.content` - which is the content of the snippet, `snippet.score` - which is the calculated relevance score of the snippet in the range [0, 1] and `snippet.reference`.

Reference has two attributes: `snippet.reference.pages` and `snippet.reference.file`, the latter has the same file information like [Describe File](https://docs.pinecone.io/reference/api/assistant/describe_file)



## 3. Use Assistant context with an LLM
In this demonstration we will show how to use Pinecone Assistant Context API with Anthropic's Claude model to generate an answer. This can use any model you'd like: proprietary, open-source, local or as-a-service

### Install Anthropic SDK

In [16]:
%pip install -qU \
     anthropic==0.54.0

Note: you may need to restart the kernel to use updated packages.


### Set the Anthropic API key

Next, we'll need to get a [Claude API key](https://docs.anthropic.com/en/api/overview). The code below will prompt you to enter it and then set it in the environment.

In [17]:
import anthropic

def get_anthropic_api_key():
    """
    Get Anthropic API key from environment variable or prompt user for input.
    Returns the API key as a string.
    """

    api_key = os.environ.get("ANTHROPIC_API_KEY")
    
    if api_key is None:
        print("Anthropic API key not found in environment.")
        try:
            api_key = getpass("Please enter your Anthropic API key: ")
            # Save to environment for future use in session
            os.environ["ANTHROPIC_API_KEY"] = api_key
            print("Anthropic API key saved to environment.")
        except Exception as e:
            print(f"Error getting Anthropic API key: {e}")
            return None
    
    return api_key

ANTHROPIC_API_KEY = get_anthropic_api_key()

Anthropic API key not found in environment.
Anthropic API key saved to environment.


### Create the client

In [18]:
anthropic_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    # or uncomment and add the API key inline
    # api_key = "my_api_key"
)

### Prepare prompts

Using the first 5 snippets as context, we'll prepare a prompt with the user query to send to Anthropic.

In [19]:
# Let's use the first 5 snippets as our context

contents = []

for snippet in response.snippets[:5]:
  contents.append(
      {
          "type": snippet.type,
          "content": snippet.content,
          "score": snippet.score,
          "file_name": snippet.reference.file.name,
          "file_page": snippet.reference.pages
      }
)

In [20]:
system_prompt = """
  you are a helpful knowledge assistant, use the context provided
  in the <context></context> tags to answer the question provided in
  the <question></question> tags. Keep you answers concise please
""".strip()

question = "Who is the Chief Financial Officer of Netflix?"

user_message = f"""
  answer the following question using the context:

  <context>{contents}</context>

  <question>{question}</question>
"""

### Use Claude to generate a response

In [21]:
message = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20241022",
    system=system_prompt,
    max_tokens=1024,
    messages=[
        {"role": "user", "content": user_message}
    ]
)
print(message.content[0].text)

According to the context, David Wells is the Chief Financial Officer (CFO) of Netflix as of January 28, 2016.


## 5. Validate results with Assistant Chat

The Assistant Chat endpoint provides end-to-end answers to question..

In [22]:
# create a single message query
message = [Message(role= "user", content ="Who is the Chief Financial Officer of Netflix?")]

chat_resp = assistant.chat(messages=message, model='claude-3-5-sonnet')

# Print the assistant response
answer = chat_resp.message.content
print(answer)

According to the search results, the Chief Financial Officer of Netflix is David Wells. This can be seen in the signature line of Exhibit 31.2, which is a certification signed by "David Wells, Chief Financial Officer".


The response also contains the citations:

In [23]:
citations = chat_resp.citations

filename = citations[0]['references'][0]['file']['name']
pages = citations[0]['references'][0]['pages']
# print the files and the pages
print(f"The answer was generated using file: {filename} pages: {pages}")

The answer was generated using file: netflix-10k.pdf pages: [66, 67, 68, 69, 70]


## Cleanup Resources

Run this when you're done experimenting to delete the Assistant.

In [ ]:
pc.assistant.delete_assistant(
    assistant_name="my-cool-context-assistant", 
)